In [9]:
pip install streamlit


     ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
     ---------------------------------------- 44.3/44.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.4/10.0 MB 7.4 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.0 MB 9.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.4/10.0 MB 10.2 MB/s eta 0:00:01
   ------- -------------------------------- 2.0/10.0 MB 10.6 MB/s eta 0:00:01
   ---------- ----------------------------- 2.5/10.0 MB 10.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.1/10.0 MB 11.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.6/10.0 MB 11.7 MB/s eta 0:00:01
   ---------------- ----------------------- 4.2/10.0 MB 11.6 MB/s eta 0:00:01
   ----------------- ---------------------- 4.4/10.0 MB 10.7 MB/s eta 0:00:01
   ------------------- -------------------- 4.8/10.0 MB 10.6 MB/s eta 0:00:01
   ----


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\intel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:

pip install pandas


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.2/11.3 MB 7.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/11.3 MB 7.7 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/11.3 MB 5.7 MB/s eta 0:00:02
   --- ------------------------------------ 1.1/11.3 MB 6.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.7/11.3 MB 7.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/11.3 MB 8.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.8/11.3 MB 8.9 MB/s eta 0:00:01
   ----------- ---------------------------- 3.3/11.3 MB 9.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/11.3 MB 9.5 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.3 MB 10.1 MB/s eta 0:00:01
   ----


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\intel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
pip install sqlite3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\intel\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import sqlite3
from contextlib import closing
from datetime import datetime
import pandas as pd
import streamlit as st

In [12]:
DB_PATH = "leads.db"

STATUSES = ["New", "Contacted", "Qualified", "In Progress", "Won", "Lost", "Closed"]
SOURCES = ["Website", "Referral", "Email", "Phone", "Social", "Event", "Other"]


In [13]:
#database setup

In [15]:
def init_db():
    with closing(sqlite3.connect(DB_PATH)) as conn:
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS leads (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                email TEXT,
                phone TEXT,
                company TEXT,
                source TEXT,
                owner TEXT,
                status TEXT DEFAULT 'New',
                value REAL DEFAULT 0,
                tags TEXT,
                notes TEXT,
                created_at TEXT NOT NULL,
                updated_at TEXT NOT NULL
            );
            """
        )
        conn.commit()

def _now():
    return datetime.utcnow().isoformat(timespec="seconds")

def add_lead(data: dict):
    with closing(sqlite3.connect(DB_PATH)) as conn:
        cur = conn.cursor()
        cur.execute(
            """
            INSERT INTO leads (name, email, phone, company, source, owner, status, value, tags, notes, created_at, updated_at)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                data.get("name"),
                data.get("email"),
                data.get("phone"),
                data.get("company"),
                data.get("source"),
                data.get("owner"),
                data.get("status", "New"),
                float(data.get("value") or 0),
                data.get("tags"),
                data.get("notes"),
                _now(),
                _now(),
            ),
        )
        conn.commit()

def update_lead(lead_id: int, updates: dict):
    if not updates:
        return
    set_clause = ", ".join([f"{k}=?" for k in updates.keys()])
    params = list(updates.values()) + [_now(), lead_id]
    with closing(sqlite3.connect(DB_PATH)) as conn:
        conn.execute(
            f"UPDATE leads SET {set_clause}, updated_at=? WHERE id=?",
            params,
        )
        conn.commit()

def delete_lead(lead_id: int):
    with closing(sqlite3.connect(DB_PATH)) as conn:
        conn.execute("DELETE FROM leads WHERE id=?", (lead_id,))
        conn.commit()

def fetch_leads(filters: dict = None) -> pd.DataFrame:
    filters = filters or {}
    clauses = []
    params = []
    if filters.get("q"):
        q = filters["q"]
        clauses.append("(name LIKE ? OR email LIKE ? OR company LIKE ? OR owner LIKE ? OR tags LIKE ? OR notes LIKE ?)")
        like = f"%{q}%"
        params += [like] * 6
    if filters.get("status"):
        clauses.append("status = ?")
        params.append(filters["status"])
    if filters.get("owner"):
        clauses.append("owner LIKE ?")
        params.append(f"%{filters['owner']}%")
    if filters.get("source"):
        clauses.append("source = ?")
        params.append(filters["source"])

    where = f"WHERE {' AND '.join(clauses)}" if clauses else ""
    order = filters.get("order_by") or "created_at DESC"
    with closing(sqlite3.connect(DB_PATH)) as conn:
        df = pd.read_sql_query(f"SELECT * FROM leads {where} ORDER BY {order}", conn, params=params)
    return df

def export_csv(df: pd.DataFrame) -> bytes:
    return df.to_csv(index=False).encode("utf-8")

def import_csv(file) -> int:
    df = pd.read_csv(file)
    if "name" not in df.columns:
        raise ValueError("CSV must include at least a 'name' column")

    with closing(sqlite3.connect(DB_PATH)) as conn:
        cur = conn.cursor()
        count = 0
        for _, row in df.iterrows():
            cur.execute(
                """
                INSERT INTO leads (name, email, phone, company, source, owner, status, value, tags, notes, created_at, updated_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """,
                (
                    row.get("name"),
                    row.get("email"),
                    row.get("phone"),
                    row.get("company"),
                    row.get("source"),
                    row.get("owner"),
                    row.get("status") if str(row.get("status")) in STATUSES else "New",
                    float(row.get("value") or 0),
                    row.get("tags"),
                    row.get("notes"),
                    _now(),
                    _now(),
                ),
            )
            count += 1
        conn.commit()
    return count

In [16]:
#streamlite ui

In [19]:
st.set_page_config(page_title="Lead CRM", layout="wide")
init_db()

st.title("🗂️ Lead Management CRM (Python)")

2025-09-01 15:19:51.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:19:51.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:19:51.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:19:51.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [21]:
#sidebar add lead

In [23]:
with st.sidebar:
    st.header("➕ Add Lead")
    with st.form("add_form"):
        name = st.text_input("Name*")
        email = st.text_input("Email")
        phone = st.text_input("Phone")
        company = st.text_input("Company")
        col1, col2 = st.columns(2)
        with col1:
            source = st.selectbox("Source", SOURCES)
        with col2:
            owner = st.text_input("Owner")
        col3, col4 = st.columns(2)
        with col3:
            status = st.selectbox("Status", STATUSES)
        with col4:
            value = st.number_input("Deal Value", min_value=0.0, step=100.0)
        tags = st.text_input("Tags (comma-separated)")
        notes = st.text_area("Notes")
        submitted = st.form_submit_button("Add Lead")
        if submitted:
            if not name.strip():
                st.error("Name is required.")
            else:
                add_lead({
                    "name": name, "email": email, "phone": phone, "company": company,
                    "source": source, "owner": owner, "status": status,
                    "value": value, "tags": tags, "notes": notes
                })
                st.success(f"Lead '{name}' added.")

2025-09-01 15:21:33.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:21:33.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
#tabs

In [27]:
tab1, tab2, tab3 = st.tabs(["Leads", "Analytics", "Import/Export"])

2025-09-01 15:22:22.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:22:22.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:22:22.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:22:22.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [28]:
#leads tab

In [30]:
with tab1:
    st.subheader("All Leads")
    q = st.text_input("Search")
    col1, col2, col3, col4 = st.columns(4)
    with col1: status_f = st.selectbox("Status", [""] + STATUSES)
    with col2: owner_f = st.text_input("Owner filter")
    with col3: source_f = st.selectbox("Source", [""] + SOURCES)
    with col4: order_by = st.selectbox("Order", ["created_at DESC","value DESC","value ASC","name ASC","name DESC"])

    df = fetch_leads({"q": q, "status": status_f or None, "owner": owner_f, "source": source_f or None, "order_by": order_by})
    st.write(f"{len(df)} lead(s) found")
    if df.empty:
        st.info("No leads yet.")
    else:
        st.dataframe(df)

2025-09-01 15:23:12.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:12.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [31]:
#analytics tab

In [32]:
with tab2:
    st.subheader("Analytics")
    df_all = fetch_leads()
    if df_all.empty:
        st.info("No data for analytics.")
    else:
        st.bar_chart(df_all["status"].value_counts())
        st.bar_chart(df_all.groupby("status")["value"].sum())

2025-09-01 15:23:35.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:35.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:35.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:35.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:35.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:23:35.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [33]:
#import and export tab

In [34]:
with tab3:
    st.subheader("Export")
    df_all = fetch_leads()
    if not df_all.empty:
        st.download_button("⬇️ Download CSV", df_all.to_csv(index=False), "leads.csv")

    st.subheader("Import")
    file = st.file_uploader("Upload CSV", type=["csv"])
    if file:
        try:
            new = pd.read_csv(file)
            with closing(sqlite3.connect(DB_PATH)) as conn:
                new.to_sql("leads", conn, if_exists="append", index=False)
            st.success("CSV imported successfully")
        except Exception as e:
            st.error(f"Import failed: {e}")

2025-09-01 15:24:17.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-01 15:24:17.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar